In [ ]:
# importante agregar el uuid

In [1]:

import logging
import mlflow
import mlflow.sklearn
import pandas as pd

from datetime import datetime
from mlflow import MlflowClient
from mlflow.exceptions import MlflowException

from src.config.config import load_catalog

d:\desarrollos\End2end\END2END\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
client = MlflowClient()
widget = 'train'

2026/02/18 14:01:56 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.schemas
2026/02/18 14:01:56 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.tables
2026/02/18 14:01:56 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.types
2026/02/18 14:01:56 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.constraints
2026/02/18 14:01:56 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.defaults
2026/02/18 14:01:56 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.comments
2026/02/18 14:01:57 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/02/18 14:01:57 INFO alembic.runtime.migration: Will assume non-transactional DDL.


In [ ]:
catalog, params = load_catalog(env="base")

date = datetime.now()

alias = params['aliases_champion']
nombre_usecase = params['nombre_usecase']

# features
num_features = params['num_features']
cat_features = params['cat_features']

if widget == "train":

    df = catalog.load('titanic_intermediate')
    model = mlflow.sklearn.load_model(f"models:/{nombre_usecase}@champion")
    model_version = client.get_model_version_by_alias(name=nombre_usecase,alias=alias)
    
    
    # Df's
    X = df[num_features + cat_features]
    y = df[params['target']]

    predicciones = model.predict(X)

    X['prediction'] =  predicciones
    X['label'] = y

    X['version'] = model_version.version
    X['timestamp'] = date 

    catalog.save(ds_name='baseline_modelo',data=X)

elif widget == "retraining":
    df = catalog.load('retraining')
    model = mlflow.sklearn.load_model(f"models:/{nombre_usecase}@challenger") #------------el Aliases se puede cambiar------------
    X = df[num_features + cat_features]
    y = df[params['target']]

    X['version'] = model.version
    X['timestamp'] = date 

    catalog.save(ds_name='baseline_modelo',data=X)
else:
    raise ValueError("Widget debe ser 'train' o 'retraining'")

D:\DATOS\Users\darodriguezo\AppData\Local\Temp\1\ipykernel_25580\1217658206.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['prediction'] =  predicciones
D:\DATOS\Users\darodriguezo\AppData\Local\Temp\1\ipykernel_25580\1217658206.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['label'] = y
D:\DATOS\Users\darodriguezo\AppData\Local\Temp\1\ipykernel_25580\1217658206.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_i